### **DST 2(DATA FOR ALL TEAMS.csv)**
##### Trina De

In [1]:
!pip install folium==0.2.1

     |████████████████████████████████| 71kB 3.2MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=3cc7771ef271175d6bb9c548c61a7f6d82952873b58884266d2afa2c7926c672
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [2]:
!pip install coverage==3.7.1

In [3]:
!pip install pycm

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 522kB 13.6MB/s 
     |████████████████████████████████| 215kB 60.4MB/s 
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 4.5.4 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 4.5.4 which is incompatible.
  Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1


In [4]:
!pip install scikit-tda

     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
  Created wheel for ripser: filename=ripser-0.4.1-cp36-cp36m-linux_x86_64.whl size=423270 sha256=8e05b470e75c4e69393b8f773ec7d4c1d5011eefbf3e0bb3a6df5ac5d7a103c8
  Stored in directory: /root/.cache/pip/wheels/d4/12/da/56d55c3af62ebd5e0684d521f3e58c1a85ac312502c9e2d47d
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18091 sha256=28b568c25112e5d9406469a30375b58a5311215f869472c0a21835e0807cda6f
  Stored in directory: /root/.cache/pip/wheels/2b/e1/c9/1993c7f7f114b7d3fb2d3e895e02157a7ebf554861e9e54e01
Successfully built ripser hopcroftkarp


In [5]:
!pip install kmapper

##### **Packages Used**

In [0]:
import numpy as np
import pandas as pd
import time
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.stats import spearmanr
import pycm
from pycm import *
import kmapper as km
from sklearn.cluster import DBSCAN
import sklearn
from sklearn import ensemble
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

##### **Loading Notebook from Google Drive**

In [7]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!cd /gdrive/My\ Drive/

In [80]:
missing_values = ["n/a", "na", "--", ' ', "NAN","nan","NA","NaN"]
data = pd.read_excel("/gdrive/My Drive/DATA FOR ALL TEAMS-25MB.xlsx",na_values=missing_values)
data.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
0,DE2,DE3,DE4,DE5,DE6,DE7,DE9,DE14,DE16,DE18,DE19,DE22,DE32,DE38,DE39,DE40,DE41,DE42,DE43,DE48,Private Data 48-1,Private Data 48-2,DE49,DE50,DE51,DE61,DE4801,DE4801,DE4803,DE4804,DE4805,DE4806,DE4807,DE4808,DE4809,DE4810
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,NaN
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,NaN
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,NaN
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,NaN


In [86]:
data2 = data.iloc[1:,:]
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,NaN
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,NaN
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,NaN
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,NaN
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,NaN


##### **Changing non NaNs in 'TARGET' column to 'Not Fraud'**

In [0]:
data2['TARGET'] = [x if x == 'Fraud' else 'Not Fraud' for x in list(data2['TARGET'])]
set(data2['TARGET'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'Fraud', 'Not Fraud'}

In [0]:
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,Not Fraud
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,Not Fraud
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,Not Fraud
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,Not Fraud
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,Not Fraud


##### **Splitting up categorical check and encoding function into three parts**


In [0]:
# Function that only checks categorical
def check_categorical(l):
   if(len(set(l))==(len(l))):
     return False
   elif(len(set(l))==1):
     return False
   else :
     return True

In [0]:
# Function that assigns numeric encoding to it
def int_encoding(col):
    # l = np.array(l)
    # c=0
    # for i in set(l):
    #   l[np.where(l == i)] = c
    #   c=c+1
    # return l
    # cw_bins = np.arange(0,len(set(l)),1)
    # l = np.digitize(l, bins=cw_bins)
    new_col = pd.Categorical(col, categories=col.unique()).codes
    return new_col


In [0]:
# Function that normalizes the encoding to between 0 and 1
def normalized_encoding(col):
    # l = np.array(l)
    # c=0
    # for i in set(l):
    #   l[np.where(l == i)] = c
    #   c=c+1
    # l = l/np.max(l)
    # return l
    # cw_bins = np.arange(0,len(set(l)),1)
    # l = np.digitize(l, bins=cw_bins)
    # l = l/np.max(l)
    new_col = pd.Categorical(col, categories=col.unique()).codes
    new_col = new_col/np.max(list(new_col))
    return new_col

In [0]:
# Function that returns one hot encoding of a list of a categorical variable
def one_hot_encoding(l):
    l = pd.Series(l)
    one_hot = pd.get_dummies(l)
    return one_hot ## This output is a dataframe

##### All 3 encoding functions would need the check of categorical to be performed first

##### **Finding categorical variables for 'all_txns.csv'**

##### Removing NaN type columns

In [87]:
names = list(data2.columns)
removable_indices =[]
for col in data2.columns:
  ## Removing columns that have NaNs
  if(data2[col].isnull().sum()==len(data2[col])): ## We remove those columns only which are full of nans
    removable_indices.append(col)
  elif(data2[col].isnull().sum()<len(data2[col])):
    data2[col] = data2[col].fillna('empty') ## changing NaNs to 'empty' so they can represent a seperate category
  ## Removing columns that have object data type such as date object or filled with '???'
  # if(data2[col].dtype.name == 'object'): ## Turns out we cannot remove object data types here because all the data types are object and removing them would remove everything
  #   removable_indices.append(col)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
removable_indices = set(removable_indices)
data2 = data2.drop(removable_indices,1)

In [89]:
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,empty,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,empty,empty,empty,356,356,356,210201210141,IN1,ATM,0917,empty,empty,empty,35000,empty,empty,empty
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,empty,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,empty,empty,empty,356,356,356,210201210141,IN1,ATM,0796,empty,empty,empty,44000,empty,empty,empty
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,empty,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,empty,empty,empty,356,356,356,210201210141,IN1,ATM,0796,empty,empty,empty,42000,empty,empty,empty
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,empty,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,empty,empty,empty,144,840,356,210201210141,IN1,ATM,0797,empty,empty,empty,14791.4,empty,empty,empty
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,empty,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,empty,empty,empty,356,840,356,200111100040,IN1,POS,0797,empty,empty,empty,177495,empty,empty,empty


In [90]:
data2.shape

(996861, 36)

In [0]:
all_teams_data_encoded = pd.DataFrame()
all_teams_data_normalized_encoded = pd.DataFrame()


In [92]:
start_time = time.time()
names = list(data2.columns)
for col in data2.columns:
  print(col)
  bool = check_categorical(list(data2[col]))
  if(bool == True):
    ## Changing categorical columns to encoded columns
    all_teams_data_encoded[col] = int_encoding(data2[col])
    all_teams_data_normalized_encoded[col] = normalized_encoding(data2[col])
#data2.head()
end_time = time.time()
print("Time taken to find and encode categorical columns : {} seconds".format(end_time-start_time))

Card-Wallet-Token ID
PROCESSING_CODE
TRANSACTION_AMOUNT
SETTLEMENT_AMOUNT
BILLING_AMOUNT
Transaction Date and Time
CONVERSION_RATE
Expiry Date
CONVERSION_RATE_DATE
Merchant Category Code
ACQUIRING_COUNTRY_CODE
POS entry Mode
ACQUIRER_INSTITUTION_CODE
Authorization Code
Approval Decision
Card Service Code
TID
MID
MERCHANT NAME AND ADDRESS
Private Data 48
Transaction Category Code
DE48 Data Length
TRANSACTION_CURRENCY
SETTLEMENT_CURRENCY
BILLING_CURRENCY
POS_DATA
Interface ID
Terminal Type
Token Data-C1
Additional Payment Info
Ecommerce Data
ECI Indicator
Remaining Balance
Private Data - Additional Info
Token Data
TARGET
Time taken to find and encode categorical columns : 14.563957452774048 seconds


##### Time taken is around 8 seconds

In [93]:
all_teams_data_encoded.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,1,0,2,2,2,2,0,1,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0
3,2,0,3,3,3,3,1,0,0,0,1,0,0,3,0,0,2,1,2,0,0,0,1,1,0,0,0,0,2,0,0,0,3,0,0,0
4,3,1,4,4,4,4,0,1,0,1,2,0,1,4,0,0,3,2,3,0,0,0,0,1,0,1,0,1,2,0,0,0,4,0,0,0


In [94]:
all_teams_data_normalized_encoded.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000032,0.000000,0.000083,0.000072,0.000081,0.000037,0.000000,0.014925,0.0,0.000000,0.000000,0.0,0.000000,0.000026,0.0,0.0,0.000038,0.000000,0.000045,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.001377,0.0,0.0,0.0,0.000024,0.0,0.0,0.0
2,0.000032,0.000000,0.000165,0.000144,0.000162,0.000074,0.000000,0.014925,0.0,0.000000,0.000000,0.0,0.000000,0.000051,0.0,0.0,0.000038,0.000000,0.000045,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.001377,0.0,0.0,0.0,0.000048,0.0,0.0,0.0
3,0.000065,0.000000,0.000248,0.000215,0.000243,0.000110,0.003311,0.000000,0.0,0.000000,0.016949,0.0,0.000000,0.000077,0.0,0.0,0.000076,0.000053,0.000091,0.0,0.0,0.0,0.022727,0.333333,0.0,0.000000,0.0,0.000000,0.002755,0.0,0.0,0.0,0.000073,0.0,0.0,0.0
4,0.000097,0.090909,0.000330,0.000287,0.000324,0.000147,0.000000,0.014925,0.0,0.003831,0.033898,0.0,0.002577,0.000102,0.0,0.0,0.000114,0.000105,0.000136,0.0,0.0,0.0,0.000000,0.333333,0.0,0.012195,0.0,0.333333,0.002755,0.0,0.0,0.0,0.000097,0.0,0.0,0.0


In [0]:
## We are removing these since we know that amounts are continous variables and does not pertain to our analysis here
all_teams_data_encoded = all_teams_data_encoded.drop(['TRANSACTION_AMOUNT','SETTLEMENT_AMOUNT','BILLING_AMOUNT'],1)
all_teams_data_normalized_encoded = all_teams_data_normalized_encoded.drop(['TRANSACTION_AMOUNT','SETTLEMENT_AMOUNT','BILLING_AMOUNT'],1)

In [96]:
print(all_teams_data_encoded.shape)
print(all_teams_data_normalized_encoded.shape)

(996861, 33)
(996861, 33)


##### **Writing encoded dataframe to google drive to be used directly for future purposes**

In [0]:
all_teams_data_encoded.to_csv('all_teams_data_encoded.csv')
!cp all_teams_data_encoded.csv /gdrive/My\ Drive/all_teams_data_encoded.csv

In [0]:
all_teams_data_normalized_encoded.to_csv('all_teams_data_normalized_encoded.csv')
!cp all_teams_data_normalized_encoded.csv /gdrive/My\ Drive/all_teams_data_normalized_encoded.csv

##### **From now on we will be using all_teams_data_encoded.csv and all_teams_data_normalized_encoded.csv for our purposes**

In [0]:
encoded = pd.read_csv("/gdrive/My Drive/all_teams_data_encoded.csv")
normalized_encoded = pd.read_csv("/gdrive/My Drive/all_teams_data_normalized_encoded.csv")

##### **Function to remove columns which are a list on distinct values for eg. the index**

In [0]:
def remove_index_like(df):
  df_new = df.copy()
  indices = []
  for i in df_new.columns:
    if len(df[i]) == len(set(df[i])):
      indices.append(i)
  df_new = df_new.drop(indices,1)
  return df_new

In [0]:
encoded = remove_index_like(encoded)

In [110]:
encoded.shape

(996861, 33)

In [111]:
normalized_encoded = remove_index_like(normalized_encoded)
normalized_encoded.shape

(996861, 33)

##### So we end up with 33 columns here unlike the 27 columns before in all_txns.csv

##### **Function to remove constant values**


In [0]:
def remove_constants(df):
  df_new = df.copy()
  indices = []
  for col in df_new.columns:
    if len(set(df_new[col])) == 1:
      indices.append(col)
  df_new = df_new.drop(indices,1)
  return df_new

In [112]:
encoded = remove_constants(encoded)
encoded.shape

(996861, 33)

In [113]:
normalized_encoded = remove_constants(normalized_encoded)
normalized_encoded.shape

(996861, 33)

##### **Correlation between columns of encoded data**

##### **Spearman's Rank Correlation**

In [114]:
corr_spearman_df = encoded.corr(method='spearman')
corr_spearman_df

,Card-Wallet-Token ID,PROCESSING_CODE,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
Card-Wallet-Token ID,1.000000,0.999585,0.999913,0.999318,0.999327,-0.155218,0.999255,0.999615,0.999785,0.999590,-0.962031,0.999697,0.999610,0.999760,-0.961053,0.999751,-0.367940,-0.367943,-0.364666,0.999643,0.999696,0.999663,0.999615,0.999797,0.999668,0.663326,-0.348935,-0.511133,-0.536157,-0.946997,-0.348937,-0.040023,-0.051839
PROCESSING_CODE,0.999585,1.000000,0.999582,0.999820,0.999603,-0.162340,0.999672,0.999873,0.999782,0.999599,-0.961977,0.999863,0.999735,0.999520,-0.961516,0.999516,-0.382416,-0.382418,-0.378985,0.999895,0.999861,0.999912,0.999721,0.999798,0.999917,0.664028,-0.364290,-0.502114,-0.527206,-0.946354,-0.364290,-0.040079,-0.052523
Transaction Date and Time,0.999913,0.999582,1.000000,0.999343,0.999331,-0.157678,0.999261,0.999617,0.999760,0.999566,-0.961675,0.999668,0.999580,0.999774,-0.961035,0.999751,-0.369354,-0.369352,-0.366088,0.999643,0.999666,0.999663,0.999584,0.999772,0.999667,0.663294,-0.350413,-0.511182,-0.536185,-0.946404,-0.350412,-0.040058,-0.052054
CONVERSION_RATE,0.999318,0.999820,0.999343,1.000000,0.999564,-0.168569,0.999609,0.999852,0.999508,0.999339,-0.961708,0.999741,0.999397,0.999335,-0.961846,0.999324,-0.394646,-0.394647,-0.391078,0.999867,0.999740,0.999857,0.999383,0.999522,0.999857,0.664204,-0.377431,-0.503498,-0.527648,-0.945469,-0.377431,-0.039746,-0.052823
Expiry Date,0.999327,0.999603,0.999331,0.999564,1.000000,-0.163129,0.999344,0.999639,0.999528,0.999333,-0.962281,0.999610,0.999388,0.999328,-0.962053,0.999330,-0.382335,-0.382334,-0.378897,0.999653,0.999607,0.999666,0.999390,0.999530,0.999666,0.664020,-0.364459,-0.506650,-0.531037,-0.946465,-0.364459,-0.039889,-0.051194
CONVERSION_RATE_DATE,-0.155218,-0.162340,-0.157678,-0.168569,-0.163129,1.000000,-0.164201,-0.162483,-0.155810,-0.155739,0.156625,-0.155104,-0.156176,-0.155858,0.168425,-0.155295,0.426909,0.426609,0.430905,-0.162363,-0.155280,-0.162304,-0.155806,-0.155889,-0.162293,-0.112366,0.448327,-0.003764,-0.003948,-0.007038,0.447078,-0.000297,0.092702
Merchant Category Code,0.999255,0.999672,0.999261,0.999609,0.999344,-0.164201,1.000000,0.999659,0.999483,0.999340,-0.962394,0.999603,0.999537,0.999129,-0.962126,0.999132,-0.384126,-0.384131,-0.380685,0.999656,0.999601,0.999666,0.999527,0.999486,0.999669,0.663798,-0.367217,-0.497187,-0.521198,-0.946536,-0.367217,-0.040288,-0.051747
ACQUIRING_COUNTRY_CODE,0.999615,0.999873,0.999617,0.999852,0.999639,-0.162483,0.999659,1.000000,0.999839,0.999668,-0.963366,0.999923,0.999708,0.999594,-0.963213,0.999616,-0.380490,-0.380491,-0.377057,0.999981,0.999922,0.999966,0.999712,0.999823,0.999963,0.663871,-0.363823,-0.507555,-0.530500,-0.947785,-0.363823,-0.040072,-0.050166
POS entry Mode,0.999785,0.999782,0.999760,0.999508,0.999528,-0.155810,0.999483,0.999839,1.000000,0.999788,-0.963486,0.999911,0.999837,0.999765,-0.962635,0.999785,-0.368034,-0.368034,-0.364761,0.999862,0.999907,0.999880,0.999840,0.999988,0.999880,0.663253,-0.348998,-0.510036,-0.534966,-0.948491,-0.348998,-0.040253,-0.050496
ACQUIRER_INSTITUTION_CODE,0.999590,0.999599,0.999566,0.999339,0.999333,-0.155739,0.999340,0.999668,0.999788,1.000000,-0.962282,0.999694,0.999649,0.999568,-0.961075,0.999633,-0.366878,-0.366878,-0.363598,0.999668,0.999696,0.999666,0.999672,0.999800,0.999671,0.663295,-0.348943,-0.510941,-0.534539,-0.947231,-0.348942,-0.040108,-0.051887


##### **We will now try to return the pairs for which there is significant correlation between columns**

In [0]:
index_dict2={}
for col in corr_spearman_df:
  ## Picking out pairs for which the correlation is greater than 0.7 or by our understanding, high
  index = np.where(abs(corr_spearman_df[col]) > 0.7)
  ## Getting indices of such pairs
  index_dict2[col]=index

In [0]:
names2=np.array(corr_spearman_df.columns)
for key in index_dict2.keys():
  ## Coverting the indices into column names
  index_dict2[key]=names2[index_dict2[key][0]]

In [117]:
index_dict2

{'ACQUIRER_INSTITUTION_CODE': array(['Card-Wallet-Token ID', 'PROCESSING_CODE',
        'Transaction Date and Time', 'CONVERSION_RATE', 'Expiry Date',
        'Merchant Category Code', 'ACQUIRING_COUNTRY_CODE',
        'POS entry Mode', 'ACQUIRER_INSTITUTION_CODE',
        'Authorization Code', 'Approval Decision', 'Card Service Code',
        'TID', 'MID', 'MERCHANT NAME AND ADDRESS', 'TRANSACTION_CURRENCY',
        'SETTLEMENT_CURRENCY', 'BILLING_CURRENCY', 'POS_DATA',
        'Interface ID', 'Terminal Type', 'Remaining Balance'], dtype=object),
 'ACQUIRING_COUNTRY_CODE': array(['Card-Wallet-Token ID', 'PROCESSING_CODE',
        'Transaction Date and Time', 'CONVERSION_RATE', 'Expiry Date',
        'Merchant Category Code', 'ACQUIRING_COUNTRY_CODE',
        'POS entry Mode', 'ACQUIRER_INSTITUTION_CODE',
        'Authorization Code', 'Approval Decision', 'Card Service Code',
        'TID', 'MID', 'MERCHANT NAME AND ADDRESS', 'TRANSACTION_CURRENCY',
        'SETTLEMENT_CURRENCY', 'BILL

##### **Doing the same for the normalized and encoded dataframe**

In [118]:
start = time.time()
corr_spearman_df_normed = normalized_encoded.corr(method='spearman')

index_dict3={}
for col in corr_spearman_df_normed:
  index = np.where(abs(corr_spearman_df_normed[col]) > 0.7)
  index_dict3[col]=index

names3=np.array(corr_spearman_df_normed.columns)
for key in index_dict3.keys():
  index_dict3[key]=names3[index_dict3[key][0]]

end = time.time()
print("Time taken to create Correlation Dictionary for encoded and normalized dataframe is: {} seconds".format(end-start))

Time taken to create Correlation Dictionary for encoded and normalized dataframe is: 35.93488383293152 seconds


In [119]:
index_dict3

{'ACQUIRER_INSTITUTION_CODE': array(['Card-Wallet-Token ID', 'PROCESSING_CODE',
        'Transaction Date and Time', 'CONVERSION_RATE', 'Expiry Date',
        'Merchant Category Code', 'ACQUIRING_COUNTRY_CODE',
        'POS entry Mode', 'ACQUIRER_INSTITUTION_CODE',
        'Authorization Code', 'Approval Decision', 'Card Service Code',
        'TID', 'MID', 'MERCHANT NAME AND ADDRESS', 'TRANSACTION_CURRENCY',
        'SETTLEMENT_CURRENCY', 'BILLING_CURRENCY', 'POS_DATA',
        'Interface ID', 'Terminal Type', 'Remaining Balance'], dtype=object),
 'ACQUIRING_COUNTRY_CODE': array(['Card-Wallet-Token ID', 'PROCESSING_CODE',
        'Transaction Date and Time', 'CONVERSION_RATE', 'Expiry Date',
        'Merchant Category Code', 'ACQUIRING_COUNTRY_CODE',
        'POS entry Mode', 'ACQUIRER_INSTITUTION_CODE',
        'Authorization Code', 'Approval Decision', 'Card Service Code',
        'TID', 'MID', 'MERCHANT NAME AND ADDRESS', 'TRANSACTION_CURRENCY',
        'SETTLEMENT_CURRENCY', 'BILL

##### **Pearson's Correlation**

In [120]:
start = time.time()
corr_pearson_df = encoded.corr(method='pearson')

index_dict={}
for col in corr_pearson_df:
  ## picking out columns for which correlation with col is greater than 0.7
  index = np.where(abs(corr_pearson_df[col]) > 0.7)
  index_dict[col]=index

names=np.array(corr_pearson_df.columns)
for key in index_dict.keys():
  index_dict[key]=names[index_dict[key][0]]

end = time.time()
print("Time taken to create Correlation Dictionary for only encoded dataframe is: {} seconds".format(end-start))


Time taken to create Correlation Dictionary for only encoded dataframe is: 2.488689422607422 seconds


In [121]:
index_dict

{'ACQUIRER_INSTITUTION_CODE': array(['Card-Wallet-Token ID', 'PROCESSING_CODE',
        'Transaction Date and Time', 'CONVERSION_RATE', 'Expiry Date',
        'Merchant Category Code', 'ACQUIRING_COUNTRY_CODE',
        'POS entry Mode', 'ACQUIRER_INSTITUTION_CODE',
        'Authorization Code', 'Approval Decision', 'Card Service Code',
        'TID', 'MERCHANT NAME AND ADDRESS', 'TRANSACTION_CURRENCY',
        'SETTLEMENT_CURRENCY', 'BILLING_CURRENCY', 'POS_DATA',
        'Interface ID', 'Terminal Type', 'Remaining Balance'], dtype=object),
 'ACQUIRING_COUNTRY_CODE': array(['Card-Wallet-Token ID', 'PROCESSING_CODE',
        'Transaction Date and Time', 'CONVERSION_RATE', 'Expiry Date',
        'Merchant Category Code', 'ACQUIRING_COUNTRY_CODE',
        'POS entry Mode', 'ACQUIRER_INSTITUTION_CODE',
        'Authorization Code', 'Approval Decision', 'Card Service Code',
        'TID', 'MERCHANT NAME AND ADDRESS', 'TRANSACTION_CURRENCY',
        'SETTLEMENT_CURRENCY', 'BILLING_CURRENCY',

##### This does not make sense since this are not actual numerical values, encodings rather, so we cannot consider linear correlation between them. Also the NaNs appear when there are constants in the column. They will be weeded out when we consider significant correlations and will be removed before clustering.

##### **Checking for constants once again**

In [0]:
def remove_constants(df):
  df_new = df.copy()
  indices = []
  for col in df_new.columns:
    if len(set(df_new[col])) == 1:
      indices.append(col)
  df_new = df_new.drop(indices,1)
  return df_new

In [126]:
encoded = remove_constants(encoded)
encoded.shape

(996861, 33)

##### Doing the test for all the pairs in the dataframe encoded


In [0]:
names = encoded.columns
names = sorted(names)

In [134]:
start = time.time()
related = []
pvalue = []
for i in range(0,len(names)-1):
  for j in range(i+1,len(names)):
    c=1
    n1 = names[i]
    n2 = names[j]
    df = pd.DataFrame()
    df.insert(0,n1,encoded[n1])
    df.insert(1,n2,encoded[n2])
    df_test = df.sample(1000)
    if len(set(df_test[n1])) == 1 :
      c = 0
    elif len(set(df_test[n2])) == 1 :
      c = 0
    while c == 0: ## Sampling till more than one category appears in each column
      df_test = df.sample(1000)
      if len(set(df_test[n1])) == 1 :
        c = 0
      elif len(set(df_test[n2])) == 1 :
        c = 0
      else :
        c = 1
    cm = ConfusionMatrix(list(df_test[n1]), list(df_test[n2]))
    related.append([n1,n2])
    pvalue.append(cm.PValue)
      #print("{}".fomat(n1+','+n2))
    print(i,j)
end = time.time()
print("Time taken to get categorically related variables from encoded is : {}".format(end-start))

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
4 30
4 31
4 32
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25
5 26
5 27
5 28
5 29
5 30
5 31
5 32
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
6 24
6 25
6 26
6 27
6 28
6 29
6 30
6 31
6 32
7 8
7 9
7 10
7 11
7 12
7 13

##### The time taken about is about 1.5 minutes.

In [135]:
## The following are the related columns 
related

[['ACQUIRER_INSTITUTION_CODE', 'ACQUIRING_COUNTRY_CODE'],
 ['ACQUIRER_INSTITUTION_CODE', 'Additional Payment Info'],
 ['ACQUIRER_INSTITUTION_CODE', 'Approval Decision'],
 ['ACQUIRER_INSTITUTION_CODE', 'Authorization Code'],
 ['ACQUIRER_INSTITUTION_CODE', 'BILLING_CURRENCY'],
 ['ACQUIRER_INSTITUTION_CODE', 'CONVERSION_RATE'],
 ['ACQUIRER_INSTITUTION_CODE', 'CONVERSION_RATE_DATE'],
 ['ACQUIRER_INSTITUTION_CODE', 'Card Service Code'],
 ['ACQUIRER_INSTITUTION_CODE', 'Card-Wallet-Token ID'],
 ['ACQUIRER_INSTITUTION_CODE', 'DE48 Data Length'],
 ['ACQUIRER_INSTITUTION_CODE', 'ECI Indicator'],
 ['ACQUIRER_INSTITUTION_CODE', 'Ecommerce Data'],
 ['ACQUIRER_INSTITUTION_CODE', 'Expiry Date'],
 ['ACQUIRER_INSTITUTION_CODE', 'Interface ID'],
 ['ACQUIRER_INSTITUTION_CODE', 'MERCHANT NAME AND ADDRESS'],
 ['ACQUIRER_INSTITUTION_CODE', 'MID'],
 ['ACQUIRER_INSTITUTION_CODE', 'Merchant Category Code'],
 ['ACQUIRER_INSTITUTION_CODE', 'POS entry Mode'],
 ['ACQUIRER_INSTITUTION_CODE', 'POS_DATA'],
 ['ACQUIRE

In [0]:
related_df = pd.DataFrame(columns = ['Categorical_1','Categorical_2', 'pvalue'])
Categorical_1 = []
Categorical_2 = []
for i in range(0,len(related)):
  Categorical_1.append(related[i][0])
  Categorical_2.append(related[i][1])
related_df['Categorical_1'] = Categorical_1
related_df['Categorical_2'] = Categorical_2
related_df['pvalue'] = pvalue

In [137]:
related_df.head()

,Categorical_1,Categorical_2,pvalue
0,ACQUIRER_INSTITUTION_CODE,ACQUIRING_COUNTRY_CODE,1.0
1,ACQUIRER_INSTITUTION_CODE,Additional Payment Info,1.0
2,ACQUIRER_INSTITUTION_CODE,Approval Decision,1.0
3,ACQUIRER_INSTITUTION_CODE,Authorization Code,1.0
4,ACQUIRER_INSTITUTION_CODE,BILLING_CURRENCY,1.0


In [139]:
related_df[related_df['pvalue']<1]

,Categorical_1,Categorical_2,pvalue
67,Additional Payment Info,CONVERSION_RATE_DATE,0.592548
70,Additional Payment Info,DE48 Data Length,0.592548
71,Additional Payment Info,ECI Indicator,0.999515
72,Additional Payment Info,Ecommerce Data,0.999879
81,Additional Payment Info,Private Data - Additional Info,0.628837
82,Additional Payment Info,Private Data 48,0.592548
85,Additional Payment Info,TARGET,0.697350
89,Additional Payment Info,Token Data,0.815516
91,Additional Payment Info,Transaction Category Code,0.932397
205,CONVERSION_RATE_DATE,DE48 Data Length,0.999412


##### So this method shows that none of the pairs of columns are dependent on each other.

### **Clustering using KMapper, pick clusters with more fraud data, using dataframe encoded**

In [0]:
encoded.to_csv('all_teams_data_ready_for_clustering.csv')
!cp all_teams_data_ready_for_clustering.csv /gdrive/My\ Drive/all_teams_data_ready_for_clustering.csv

In [143]:
mapper_data = pd.read_csv('/gdrive/My Drive/all_teams_data_ready_for_clustering.csv')
mapper_data.shape

(996861, 34)

In [0]:
l=[]
for i  in np.arange(1,len(mapper_data),1):
  l.append(list(mapper_data.iloc[i,:]))
l=np.array(l)

In [146]:
l

array([[     1,      1,      0, ...,      0,      0,      0],
       [     2,      1,      0, ...,      0,      0,      0],
       [     3,      2,      0, ...,      0,      0,      0],
       ...,
       [996858,  30858,     11, ...,      0,      0,      0],
       [996859,  30858,     11, ...,      0,      0,      0],
       [996860,  30858,     11, ...,      0,      0,      0]])

##### **Trials for the right parameters for clustering**

##### Here there are parameters like **nr_cubes**, inside DBScan algorithm,**eps**, **leaf_size**, **min_samples**, **metric** which are specific to problem which we try to optimize by trial. For many combinations of these parameters that are unsuitable for the dataset, the clustering would not work.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.dbscan.html

##### Look at the above link for details about arguments of DBScan

In [159]:
# Initialize
mapper = km.KeplerMapper(verbose=1)

start = time.time()

choice_indices = np.random.choice(len(l), 100000, replace=False)
new_l = [l[i] for i in choice_indices] ## We randomly select 100000 rows out of 900000 to reduce complexity
new_l = np.array(new_l)
# Fit to and transform the data
projected_data = mapper.fit_transform(new_l, projection=[0,1]) # X-Y axis

# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(projected_data, new_l, nr_cubes=10, overlap_perc = 0.1 ,clusterer = DBSCAN(algorithm ='auto',eps = 2000, leaf_size = 2000, metric='euclidean',
    metric_params=None, min_samples=200, n_jobs=-1, p=None))

# Visualize it
html=mapper.visualize(graph, path_html="all_teams_data_clustering_output.html",
                 title="all_teams_data(eps = 100, min_samples  = 10, overlap_perc = 0.1)")
end = time.time()
print("Time taken to do clustering is : {}".format(end-start))

KeplerMapper()
..Composing projection pipeline of length 1:
	Projections: [0, 1]
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (100000, 34)

..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (100000, 34) using lens shaped (100000, 2)

Creating 100 hypercubes.

Created 9 edges and 10 nodes in 0:00:15.402597.
Wrote visualization to: all_teams_data_clustering_output.html
Time taken to do clustering is : 15.674800872802734


The combination of eps, leafsize, min_samples that did not work(took too long to run) are:
 > 10, 500, 10

 > 10, 2000, 10

 > 2000, 2000, 10

##### These set of parameters do not help much giving rise to a huge number of clusters with low membership which makes it difficult to identify characteristics of clusters. There are a few edges for overlapping clusters but there those are less and not much connected, not helping significantly in inference.

##### The final set of parameters used gives one long chain with more or less uniform membership in each chain.

In [160]:
# Initialize
mapper = km.KeplerMapper(verbose=1)

start = time.time()

choice_indices = np.random.choice(len(l), 100000, replace=False)
new_l = [l[i] for i in choice_indices] ## We randomly select 100000 rows out of 900000 to reduce complexity
new_l = np.array(new_l)

# Fit to and transform the data
projected_data = mapper.fit_transform(new_l, projection=[0,1]) # X-Y axis

# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(projected_data, new_l, nr_cubes=10, overlap_perc = 0.4 ,clusterer = DBSCAN(algorithm ='auto',eps = 1000, leaf_size = 2000, metric='euclidean',
    metric_params=None, min_samples=200, n_jobs=-1, p=None))

# Visualize it
html=mapper.visualize(graph, path_html="all_teams_clustering_output2.html",
                 title="all_txns(eps = 1000 , min_samples  = 200, overlap_perc = 0.4)")
end = time.time()
print("Time taken to do clustering is : {}".format(end-start))

KeplerMapper()
..Composing projection pipeline of length 1:
	Projections: [0, 1]
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (100000, 34)

..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (100000, 34) using lens shaped (100000, 2)

Creating 100 hypercubes.

Created 93 edges and 224 nodes in 0:00:19.627704.
Wrote visualization to: all_teams_clustering_output2.html
Time taken to do clustering is : 20.43278431892395


##### For this we can see a bunch of nodes with comparatively few edges. We cannot infer much from this data, also we discover nothing significant from this shape thus telling us that we were previously closer to optimal parameters.

##### **Trying out a new projection function and clustering algorithm**

In [161]:
model = ensemble.IsolationForest(random_state=1729)
choice_indices = np.random.choice(len(l), 100000, replace=False)
new_l = [l[i] for i in choice_indices] ## We randomly select 100000 rows out of 900000 to reduce complexity
new_l = np.array(new_l)

model.fit(new_l)
lens1 = model.decision_function(new_l).reshape((new_l.shape[0], 1))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


In [162]:
start = time.time()
lens2 = mapper.fit_transform(new_l, projection="l2norm")
lens = np.c_[lens1, lens2]

graph2 = mapper.map(lens,
                      new_l,
                      nr_cubes=10,
                      overlap_perc=0.4,
                      clusterer=sklearn.cluster.KMeans(n_clusters=10,
                                                       random_state=3471))

html2=mapper.visualize(graph2, path_html="all_teams_clustering_KMeans.html",
                 title="all_teams(n_clusters = 10, overlap_perc = 0.4)")
end = time.time()
print("Time taken to do clustering is : {}".format(end-start))

..Composing projection pipeline of length 1:
	Projections: l2norm
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (100000, 34)

..Projecting data using: l2norm

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (100000, 34) using lens shaped (100000, 2)

Creating 100 hypercubes.

Created 681 edges and 320 nodes in 0:00:29.630138.
Wrote visualization to: all_teams_clustering_KMeans.html
Time taken to do clustering is : 30.63549518585205


##### This set of parameters gives rise to many nodes and edges, but we can see an interesting pattern of chains and branchings that can be study for its properties later on. The reason for such high number of nodes could be since we used the l2 norms which are mostly distinct. What is interesting is that there are some points which are isolated which can be useful in our case to study seperately.




In [163]:
# model = ensemble.IsolationForest(random_state=1729)
# model.fit(l)
# lens1 = model.decision_function(l).reshape((l.shape[0], 1))

#Since we wish to visualize we are forced to have number of components under PCA to 1
start = time.time()

choice_indices = np.random.choice(len(l), 100000, replace=False)
new_l = [l[i] for i in choice_indices] ## We randomly select 100000 rows out of 900000 to reduce complexity
new_l = np.array(new_l)

lens3 = mapper.fit_transform(new_l, projection=PCA(n_components=1))

lens13 = np.c_[lens1, lens3]

graph3 = mapper.map(lens13,
                      new_l,
                      nr_cubes=10,
                      overlap_perc=0.4,
                      clusterer=sklearn.cluster.KMeans(n_clusters=5,
                                                       random_state=3471))

html3=mapper.visualize(graph3, path_html="all_teams_clustering_KMeans2.html",
                 title="all_txns(n_clusters = 5, overlap_perc = 0.4, PCA_n_components = 1)")
end = time.time()
print("Time taken to do clustering is : {}".format(end-start))

..Composing projection pipeline of length 1:
	Projections: PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (100000, 34)

..Projecting data using: 
	PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)


..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (100000, 34) using lens shaped (100000, 2)

Creating 100 hypercubes.

Created 1333 edges and 450 nodes in 0:00:20.546594.
Wrote visualization to: all_teams_clustering_KMeans2.html
Time taken to do clustering is : 22.511289596557617


##### This produces an even more interesting result than before. Here most of the nodes have edges between them and we can see a bunch of nodes at intervals in a web like mesh. It could help to study this to find grades of risk in transactions. There could be a natural ordering found from such partitioning.  Whether this projection combined with the clustering actual helps or not can only be judged by studying the variables within each nodes and cluster of nodes.

##### **Future Work**
> 1. Try out Goodman Kruskal's Gamma and Lambda for tests of association between categorical variables.

> 2. Try changing the projection and clustering algorithm to see if better results can be achieved, better results can only be judged based on labelled data.

##### **ROUGH**

In [16]:
data2['TARGET'].fillna('Not Fraud')
set(data2['TARGET'])

{'Fraud', nan}

In [30]:
set(data2['Private Data 48'])

{nan,
 'R1805000081502228002TV7280 *h   s',
 'F1805000081502888002SG7280B r|  R',
 '? %',
 'O810728022221805000081502228002TV7280     |',
 'R810728022221805000081502228002TV7280    #',
 'R810728027742202001805000081502228002TV7280 S',
 'R1805000081502228002TV7280  O|',
 'H1805000081502228002TV7280    rY ~',
 'R1805000081502228002TV7280Y',
 'R810728022221805000081502228002TV7280 7gZ@sO!',
 'R810728022221805000081502228002TV7280Y~  ?Z',
 'R810728022221805000081502228002TV7280. g{ ( 0',
 'F1805000001502228002SG7280  {',
 'R810728022221805000081502228002SG7280<htO^ A',
 '?  !',
 'H810728022221805000081502228002TV7280 l< <m@',
 'R810728022221805000081502228002TV7280 } 2',
 'H810728022221805000081502228002TV7280z    5N',
 'F810728022221805000081502228002SG7280 +G  0',
 'R1805080081502228002TV7280       ~',
 'R1805000081502228002TV7280>?',
 'F810728022221805000081502228002SG7280  )4b:',
 '?   85',
 'R810728022221805000081502228002TV7280 2O    E',
 'R1805000081502228002TV7280     / |',
 'R8107

In [38]:
data2['TRANSACTION_AMOUNT'].dtype.name == 'object'

True

In [49]:
data2['Transaction Date and Time'].dtype.name

'object'

In [78]:
np.where(data2['CONVERSION_RATE_DATE'].isnull())[0]

0

In [79]:
data2.fillna('a')
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,Not Fraud
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,Not Fraud
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,Not Fraud
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,Not Fraud
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,Not Fraud
